In [1]:
from __future__ import annotations
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../src/")

from utils import *
from sklearn.impute import KNNImputer
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import geopandas as gpd

In [2]:
df = pd.read_csv(os.path.join("..", "data", "dataset_velo_acc_preprocess_15mo.csv"))
df = merge_post_format(df)
df = df.replace(pd.NA, np.nan)

C:\Users\coco8\AppData\Local\Temp\ipykernel_50416\2811508171.py:1: DtypeWarning: Columns (11,14,19,20,24,25,34,40,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("..", "data", "dataset_velo_acc_preprocess_15mo.csv"))


In [3]:
df["dep"] = df["dep"].astype("string")
df_paris = df.loc[df["dep"].str[:2] == "75"] #We keep only Paris accident

In [4]:
df_paris.head()

,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,...,adr,gps,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,1,0.0,3.0,0.0,2.0,2.0,4,2.0,NaN,3.0,...,PONT DU CARROUSEL,NaN,NaN,NaN,750,0.0,2.0,2.0,2009-04-30,1
3,2,0.0,2.0,0.0,2.0,1.0,4,2.0,NaN,3.0,...,"118, BOULEVARD DE MAGENT",NaN,NaN,NaN,750,8.0,2.0,1.0,2009-06-03,1
4,1,0.0,NaN,14.0,0.0,1.0,4,3.0,NaN,0.0,...,"5bis, BOULEVARD DE L HOP",NaN,NaN,NaN,750,NaN,NaN,NaN,2009-06-05,1
8,1,0.0,3.0,0.0,2.0,1.0,4,1.0,NaN,0.0,...,PLACE DE L OPERA,NaN,NaN,NaN,750,0.0,2.0,3.0,2009-07-30,1
79,1,0.0,3.0,0.0,2.0,8.0,4,2.0,NaN,0.0,...,"266, AVENUE DAUMESNIL",NaN,NaN,NaN,750,0.0,2.0,8.0,2013-11-15,1


In [5]:
print("Before imputation")
missing(df_paris)

Before imputation
grav => no missing values [0%]
senc => no missing values [0%]
catv => 3045 [24.78%]
obs => no missing values [0%]
obsm => 2 [0.02%]
choc => no missing values [0%]
catr => no missing values [0%]
circ => 81 [0.66%]
nbv => 16 [0.13%]
vosp => 48 [0.39%]
prof => 1 [0.01%]
plan => no missing values [0%]
surf => no missing values [0%]
infra => 624 [5.08%]
situ => 2 [0.02%]
lum => no missing values [0%]
int => no missing values [0%]
atm => no missing values [0%]
col => 44 [0.36%]
com => no missing values [0%]
adr => no missing values [0%]
gps => 10028 [81.61%]
lat => 7429 [60.46%]
long => 7429 [60.46%]
dep => no missing values [0%]
obs2 => 3045 [24.78%]
obsm2 => 3045 [24.78%]
choc2 => 3045 [24.78%]
date => no missing values [0%]
is_in_agg => no missing values [0%]


In [6]:
df_paris.drop(["gps"], axis=1, inplace=True)
df_paris.dropna(inplace=True)

C:\Users\coco8\AppData\Local\Temp\ipykernel_50416\911893628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paris.drop(["gps"], axis=1, inplace=True)
C:\Users\coco8\AppData\Local\Temp\ipykernel_50416\911893628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paris.dropna(inplace=True)


In [7]:
df_paris.reset_index(inplace=True)
df_paris

,index,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,...,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,28815,1,0.0,1.0,0.0,0.0,5.0,4,1.0,0.0,...,112.0,ROUTE ROYALE DE BEAUTE,4883464.0,243893.0,750,0.0,0.0,3.0,2010-10-03,1
1,50611,1,0.0,2.0,0.0,2.0,0.0,4,2.0,0.0,...,110.0,"/, RUE LA FAYETTE",4852501.0,221432.0,750,0.0,2.0,0.0,2015-08-13,1
2,51041,1,0.0,3.0,0.0,2.0,2.0,4,1.0,1.0,...,118.0,"72, RUE DOUDEAUVILLE",4853181.0,221070.0,750,0.0,0.0,7.0,2015-06-27,1
3,51050,3,0.0,2.0,0.0,2.0,8.0,4,1.0,1.0,...,117.0,"02, RUE DE THANN",4888077.0,230911.0,750,0.0,2.0,1.0,2015-10-08,1
4,54694,1,1.0,1.0,0.0,2.0,0.0,4,1.0,4.0,...,101.0,"10, BOULEVARD DE SEBASTO",4885930.0,234866.0,750,0.0,2.0,0.0,2016-03-23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,79077,1,1.0,3.0,0.0,2.0,0.0,4,2.0,2.0,...,75108,RUE D AMSTERDAM,"48,8771100000","2,3269130000",75,0.0,2.0,0.0,2021-01-13,1
3173,79096,1,1.0,4.0,0.0,2.0,8.0,4,2.0,2.0,...,75117,AVENUE DE WAGRAM,"48,8820390000","2,3010560000",75,0.0,2.0,2.0,2021-01-11,1
3174,79102,1,2.0,3.0,0.0,2.0,1.0,4,1.0,3.0,...,75110,RUE DU FAUBOURG SAINT MARTIN,"48,8838690000","2,3685920000",75,0.0,2.0,6.0,2021-01-09,1
3175,79123,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,...,75112,RUE CROZATIER,"48,8489260000","2,3800250000",75,0.0,2.0,3.0,2021-01-07,1


In [8]:
def lat_long(df):
    df["lat"] = df["lat"].astype("string").apply(lambda x: str(x.split(',')[0] + x.split(',')[1])[:8] if len(x.split(',')) > 1 else str(x.split('.')[0] + x.split('.')[1])[:8]).astype("Int32")
    df["lat"] = df["lat"].apply(lambda x: x/1000000)

    df_paris["long"] = df_paris["long"].astype("string").apply(lambda x: str(x.split(',')[0] + x.split(',')[1])[:7] if len(x.split(',')) > 1 else str(x.split('.')[0] + x.split('.')[1])[:7]).astype("Int32")
    df_paris["long"] = df_paris["long"].apply(lambda x: x/1000000)
    
    return df

In [9]:
df_paris = lat_long(df_paris)
df_paris

C:\Users\coco8\AppData\Local\Temp\ipykernel_50416\3292139920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lat"] = df["lat"].astype("string").apply(lambda x: str(x.split(',')[0] + x.split(',')[1])[:8] if len(x.split(',')) > 1 else str(x.split('.')[0] + x.split('.')[1])[:8]).astype("Int32")
C:\Users\coco8\AppData\Local\Temp\ipykernel_50416\3292139920.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lat"] = df["lat"].apply(lambda x: x/1000000)
C:\Users\coco8\AppData\Local\Temp\ipykernel_50416

,index,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,...,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,28815,1,0.0,1.0,0.0,0.0,5.0,4,1.0,0.0,...,112.0,ROUTE ROYALE DE BEAUTE,48.834640,2.438930,750,0.0,0.0,3.0,2010-10-03,1
1,50611,1,0.0,2.0,0.0,2.0,0.0,4,2.0,0.0,...,110.0,"/, RUE LA FAYETTE",48.525010,2.214320,750,0.0,2.0,0.0,2015-08-13,1
2,51041,1,0.0,3.0,0.0,2.0,2.0,4,1.0,1.0,...,118.0,"72, RUE DOUDEAUVILLE",48.531810,2.210700,750,0.0,0.0,7.0,2015-06-27,1
3,51050,3,0.0,2.0,0.0,2.0,8.0,4,1.0,1.0,...,117.0,"02, RUE DE THANN",48.880770,2.309110,750,0.0,2.0,1.0,2015-10-08,1
4,54694,1,1.0,1.0,0.0,2.0,0.0,4,1.0,4.0,...,101.0,"10, BOULEVARD DE SEBASTO",48.859300,2.348660,750,0.0,2.0,0.0,2016-03-23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,79077,1,1.0,3.0,0.0,2.0,0.0,4,2.0,2.0,...,75108,RUE D AMSTERDAM,48.877110,2.326913,75,0.0,2.0,0.0,2021-01-13,1
3173,79096,1,1.0,4.0,0.0,2.0,8.0,4,2.0,2.0,...,75117,AVENUE DE WAGRAM,48.882039,2.301056,75,0.0,2.0,2.0,2021-01-11,1
3174,79102,1,2.0,3.0,0.0,2.0,1.0,4,1.0,3.0,...,75110,RUE DU FAUBOURG SAINT MARTIN,48.883869,2.368592,75,0.0,2.0,6.0,2021-01-09,1
3175,79123,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,...,75112,RUE CROZATIER,48.848926,2.380025,75,0.0,2.0,3.0,2021-01-07,1


In [10]:
df_amenagement = gpd.read_file("../data/france-20230101.geojson")

In [11]:
df_amenagement.loc[df_amenagement["ame_g"]=="AUCUN"].head()

,id_local,id_osm,code_com_d,ame_d,regime_d,sens_d,statut_d,revet_d,code_com_g,ame_g,...,ref_geo,num_iti,local_d,local_g,access_ame,lumiere,largeur_d,largeur_g,d_service,geometry
14,geovelo_90581450_16015,90581450,16015,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,EN SERVICE,LISSE,16015,AUCUN,...,OpenStreetmap,None,CHAUSSEE,None,None,None,NaN,NaN,None,"LINESTRING (0.17274 45.64648, 0.17263 45.64651..."
16,geovelo_492304837_72181,492304837,72181,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,EN SERVICE,None,72181,AUCUN,...,OpenStreetmap,None,None,None,None,None,NaN,NaN,None,"LINESTRING (0.16681 47.97614, 0.16603 47.97593..."
20,geovelo_1079978026_72181,1079978026,72181,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,EN SERVICE,None,72181,AUCUN,...,OpenStreetmap,None,None,None,None,None,NaN,NaN,None,"LINESTRING (0.18191 48.01231, 0.18181 48.01235)"
22,geovelo_574123079_72008,574123079,72008,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,EN SERVICE,None,72008,AUCUN,...,OpenStreetmap,None,None,None,None,None,NaN,NaN,None,"LINESTRING (0.18683 47.93192, 0.18634 47.93083..."
24,geovelo_851613127_72181,851613127,72181,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,EN SERVICE,LISSE,72181,AUCUN,...,OpenStreetmap,None,CHAUSSEE,None,None,None,NaN,NaN,None,"LINESTRING (0.20236 48.01569, 0.20232 48.01570..."


In [12]:
df_amenagement = preprocessing(df_amenagement)

In [13]:
df_amenagement["code_com"] = df_amenagement["code_com"].astype("string")
df_amenagement = df_amenagement.loc[df_amenagement["code_com"].str[:2] == "75"]
df_amenagement

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
403,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513
407,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210
408,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913
409,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907
425,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114
...,...,...,...,...,...,...,...,...
255679,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756
255681,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300
255686,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840
255689,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053


In [14]:
df_amenagement.reset_index(inplace = True)
df_amenagement

,index,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,403,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513
1,407,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210
2,408,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913
3,409,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907
4,425,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114
...,...,...,...,...,...,...,...,...,...
9694,255679,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756
9695,255681,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300
9696,255686,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840
9697,255689,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053


In [15]:
np.array(df_paris.index)

array([   0,    1,    2, ..., 3174, 3175, 3176], dtype=int64)

In [16]:
df_amenagement["date_maj"] = pd.to_datetime(df_amenagement["date_maj"])

In [17]:
infrastructure = df_amenagement[['latitude_dep', 'longitude_dep']].to_numpy()
accidents = df_paris[['lat', 'long']].to_numpy()


In [18]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians, asin
from tqdm import tqdm
import warnings

warnings.simplefilter('ignore')

def distance(lat_dep, lon_dep, lat_acc, lon_acc, lat_fin, lon_fin):
    R = 6371
    lat_dep, lon_dep, lat_acc, lon_acc = map(np.radians, [lat_dep, lon_dep, lat_acc, lon_acc])
    
    dlat = lat_acc - lat_dep
    dlon = lon_acc - lon_dep
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_dep) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance1 = R * c
    
    lat_fin, lon_fin = map(np.radians, [lat_fin, lon_fin])
    
    dlat = lat_acc - lat_fin
    dlon = lon_acc - lon_fin
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_fin) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance2 = R * c

    if(distance1.min()>distance2.min()):
        
        return distance2
    else:
        return distance1

def link_dataframes(accidents, infrastructure, radius):
    # On ajoute une colonne distance qui sera utilisé pour la jointure
    infrastructure['distance'] = np.nan
    infrastructure['Index_in_accident'] = np.nan
    for i, row in tqdm(infrastructure.iterrows(), total=infrastructure.shape[0]):
        lat1, lon1 = row['latitude_dep'], row['longitude_dep']
        lat2, lon2 = row['latitude_fin'], row['longitude_fin']
        
        accidents['distance'] = distance(lat1, lon1, accidents['lat'], accidents['long'], lat2, lon2)
        distance_series = accidents.query("distance <= @radius").distance
        infrastructure.at[i, 'distance'] = distance_series.min()*1000
        infrastructure.at[i, 'Index_in_accident'] = distance_series.argmin()
    accidents.drop(["index", "distance"], inplace=True, axis=1)
    infrastructure.drop(["index"], inplace=True, axis=1)
    return infrastructure

#On utilise les dataframes accidents et infrastructure
infrastructure_with_acc = link_dataframes(df_paris, df_amenagement, 1)

100%|██████████| 9699/9699 [01:02<00:00, 155.42it/s]


In [19]:
infrastructure_with_acc

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin,distance,Index_in_accident
0,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513,168.732509,5.0
1,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210,1.471661,31.0
2,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913,30.787283,29.0
3,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907,258.316176,26.0
4,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114,72.089175,85.0
...,...,...,...,...,...,...,...,...,...,...
9694,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756,438.819186,2.0
9695,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300,362.519710,5.0
9696,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840,84.365855,6.0
9697,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053,502.526278,1.0


In [20]:
infrastructure_with_acc.loc[infrastructure_with_acc['distance']<100]['distance'].describe()

count    6790.000000
mean       36.427587
std        29.839170
min         0.011119
25%         9.079421
50%        29.724349
75%        60.256314
max        99.981298
Name: distance, dtype: float64

In [21]:
infrastructure_with_acc['Index_in_accident'] = infrastructure_with_acc['Index_in_accident'].astype("Int32")

In [22]:
df_paris['Index_in_accident'] = df_paris.index
df_paris

,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,...,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg,Index_in_accident
0,1,0.0,1.0,0.0,0.0,5.0,4,1.0,0.0,1.0,...,ROUTE ROYALE DE BEAUTE,48.834640,2.438930,750,0.0,0.0,3.0,2010-10-03,1,0
1,1,0.0,2.0,0.0,2.0,0.0,4,2.0,0.0,0.0,...,"/, RUE LA FAYETTE",48.525010,2.214320,750,0.0,2.0,0.0,2015-08-13,1,1
2,1,0.0,3.0,0.0,2.0,2.0,4,1.0,1.0,0.0,...,"72, RUE DOUDEAUVILLE",48.531810,2.210700,750,0.0,0.0,7.0,2015-06-27,1,2
3,3,0.0,2.0,0.0,2.0,8.0,4,1.0,1.0,0.0,...,"02, RUE DE THANN",48.880770,2.309110,750,0.0,2.0,1.0,2015-10-08,1,3
4,1,1.0,1.0,0.0,2.0,0.0,4,1.0,4.0,1.0,...,"10, BOULEVARD DE SEBASTO",48.859300,2.348660,750,0.0,2.0,0.0,2016-03-23,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,1,1.0,3.0,0.0,2.0,0.0,4,2.0,2.0,1.0,...,RUE D AMSTERDAM,48.877110,2.326913,75,0.0,2.0,0.0,2021-01-13,1,3172
3173,1,1.0,4.0,0.0,2.0,8.0,4,2.0,2.0,0.0,...,AVENUE DE WAGRAM,48.882039,2.301056,75,0.0,2.0,2.0,2021-01-11,1,3173
3174,1,2.0,3.0,0.0,2.0,1.0,4,1.0,3.0,1.0,...,RUE DU FAUBOURG SAINT MARTIN,48.883869,2.368592,75,0.0,2.0,6.0,2021-01-09,1,3174
3175,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,0.0,...,RUE CROZATIER,48.848926,2.380025,75,0.0,2.0,3.0,2021-01-07,1,3175


In [23]:
infrastructure_with_acc = infrastructure_with_acc.merge(df_paris, left_on="Index_in_accident", right_on="Index_in_accident")
infrastructure_with_acc

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin,distance,Index_in_accident,...,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513,168.732509,5,...,101.0,"174, RUE DE RIVOLI",48.8627,2.33525,750,0.0,2.0,8.0,2016-03-24,1
1,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.871146,2.272404,48.871153,2.272512,121.493171,5,...,101.0,"174, RUE DE RIVOLI",48.8627,2.33525,750,0.0,2.0,8.0,2016-03-24,1
2,75117,PISTE CYCLABLE,AUTRE,2022-11-23,48.886993,2.313609,48.886994,2.313677,43.135302,5,...,101.0,"174, RUE DE RIVOLI",48.8627,2.33525,750,0.0,2.0,8.0,2016-03-24,1
3,75107,AUTRE,EN AGGLOMERATION,2022-11-02,48.862286,2.319523,48.862166,2.319666,58.977216,5,...,101.0,"174, RUE DE RIVOLI",48.8627,2.33525,750,0.0,2.0,8.0,2016-03-24,1
4,75107,AUTRE,EN AGGLOMERATION,2021-09-17,48.862166,2.319666,48.862034,2.319792,75.852960,5,...,101.0,"174, RUE DE RIVOLI",48.8627,2.33525,750,0.0,2.0,8.0,2016-03-24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9694,75104,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-10-10,48.855824,2.356357,48.855879,2.356390,0.781502,217,...,110.0,"9, PLACE DE LA BATAILLE",48.8835,2.36896,750,0.0,2.0,1.0,2016-03-01,1
9695,75110,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-06-07,48.873166,2.361234,48.873165,2.361375,59.621770,217,...,110.0,"9, PLACE DE LA BATAILLE",48.8835,2.36896,750,0.0,2.0,1.0,2016-03-01,1
9696,75104,COULOIR BUS+VELO,ZONE 30,2022-06-07,48.855967,2.367190,48.855957,2.367249,5.663454,217,...,110.0,"9, PLACE DE LA BATAILLE",48.8835,2.36896,750,0.0,2.0,1.0,2016-03-01,1
9697,75102,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-09-10,48.867652,2.345089,48.867367,2.344086,6.379348,212,...,109.0,"39, BOULEVARD DE ROCHECH",48.8828,2.34444,750,0.0,2.0,0.0,2016-12-01,1


In [24]:
pd.set_option("display.max_columns", None)

In [25]:
imp = KNNImputer(n_neighbors=1)
series_date = df["date"]
df = df.drop('date', axis=1)
df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
# df = pd.concat([df, series_date])
df

ValueError: could not convert string to float: '2B096'

In [ ]:
df = one_hot_encoder(df, ['lum', 'int', 'atm', 'col', 'catr', 'circ', 'vosp','prof', 'plan', 'surf', 'infra', 'situ', 'catv', 'obs', 'obsm', 'choc','grav', 'obs2', 'obsm2', 'choc2'])
df

In [ ]:
df["date"] = series_date
df

In [ ]:
print("After imputation")
missing(df)

In [ ]:
pca = PCA()
df = df.drop('date', axis=1)
X_reduced = pca.fit_transform(scale(df))
X_reduced.shape

In [ ]:
cumulative_explained_variance = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(cumulative_explained_variance)

In [ ]:
plt.bar(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ )
plt.ylabel('Percentage of explained variance')
plt.xlabel('Number of components')
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),
         np.cumsum(pca.explained_variance_ratio_ ),
         c='red',
         label="Cumulative Explained Variance")
plt.legend(loc='upper left')

In [ ]:
inertie = []
from tqdm import tqdm

for i in tqdm(range(1, 300, 25)):
    kmeans_pca = KMeans(n_clusters = i, init = 'k-means++', random_state=42)
    kmeans_pca.fit(scale(df))
    inertie.append(kmeans_pca.inertia_)

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(range(1,300, 25), inertie, marker = 'o', linestyle = '--')
plt.xlabel('K-means with pca clustering with k cluster')
plt.ylabel('Inertie')
plt.show()